# Custom LLLM Agent

This notebook goes through how to create your own custom LLM agent.

An LLM agent consists of three parts:
    
    - Tools: The tools the agent has available to use.
    - LLMChain: The LLMChain that produces the text that is parsed in a certain way to determine which action to take.
    - OutputParser: This determines how to parse the LLMOutput into 
        
        
In this notebook we walk through how to create a custom LLM agent.

In [1]:
import sys
sys.path.append("../../../../../lang-chat/backend/")
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"

In [26]:
import copy
import logging
from typing import List, Optional, Sequence, Type

import aiohttp
import requests
from app.toolkits.open_api.openapi_utils import extract_domain
from app.toolkits.open_api.tool import OpenAPITool, get_tools_from_openapi_spec
from langchain.agents.agent_toolkits.base import BaseToolkit
from langchain.tools import BaseTool
from langchain.tools.plugin import marshal_spec
from openapi_schema_pydantic import OpenAPI
from pydantic import ValidationError

class _OpenAPIModel(OpenAPI):
    """OpenAPI Model that removes misformatted parts of the spec."""

    @classmethod
    def parse_obj(cls, obj):
        try:
            return super().parse_obj(obj)
        except ValidationError as e:
            # We are handling possibly misconfigured specs and want to do a best-effort
            # job to get a reasonable interface out of it.
            new_obj = copy.deepcopy(obj)
            for error in e.errors():
                keys = error["loc"]
                item = new_obj
                for key in keys[:-1]:
                    item = item[key]
                item.pop(keys[-1], None)
            return cls.parse_obj(new_obj)

def load_ai_plugins_and_open_api_specs(urls: List[str], aiosession: Optional[aiohttp.ClientSession] = None,
        default_headers: Optional[dict] = None,):
    info = []
    for url in urls:
        response = requests.get(url)
        plugin = response.json()
        open_api_url = plugin["api"]["url"]
        default_headers = default_headers or {}
        if "auth" in plugin and "type":
            auth_type = plugin["auth"].get("type", "none")
            if auth_type == "service_http":
                token = plugin["auth"].get("verification_tokens", {}).get("openai", "")
                if "Authorization" not in default_headers:
                    default_headers["Authorization"] = f"bearer {token}"
            elif auth_type == "none":
                pass
            else:
                logger.warning("Unsupported auth type: %s", auth_type)
        response = requests.get(open_api_url)
        open_api_spec = marshal_spec(response.text)
        spec = _OpenAPIModel.parse_obj(open_api_spec)
        info.append((plugin, spec))
    return info

In [27]:
load_ai_plugins_and_open_api_specs([
    "https://www.klarna.com/.well-known/ai-plugin.json"
])

[({'schema_version': 'v1',
   'name_for_model': 'KlarnaProducts',
   'name_for_human': 'Klarna Shopping',
   'description_for_human': 'Search and compare prices from thousands of online shops.',
   'description_for_model': 'Assistant uses the Klarna plugin to get relevant product suggestions for any shopping or product discovery purpose. Assistant will reply with the following 3 paragraphs 1) Search Results 2) Product Comparison of the Search Results 3) Followup Questions. The first paragraph contains a list of the products with their attributes listed clearly and concisely as bullet points under the product, together with a link to the product and an explanation. Links will always be returned and should be shown to the user. The second paragraph compares the results returned in a summary sentence starting with "In summary". Assistant comparisons consider only the most important features of the products that will help them fit the users request, and each product mention is brief, short

In [2]:
from app.toolkits.open_api.toolkit import OpenAPIToolkit
urls = [
    "https://www.klarna.com/.well-known/ai-plugin.json"
]
def get_toolkits(urls):
    return list(map(OpenAPIToolkit.from_plugin_url, urls))
toolkits = get_toolkits(urls)
#namespaces = "\n\n".join([tk.get_typescript_namespace() for tk in toolkits])
#print(namespaces)

In [3]:
toolkits[0].get_tools()[0]

Tool(name='KlarnaProducts___productsUsingGET', description='A single, standalone question to ask of this person. This person will use: API for fetching Klarna product information', return_direct=False, verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x104d8d250>, func=<bound method Chain.run of OpenAPIChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x104d8d250>, verbose=False, endpoint_spec='API for fetching Klarna product information\n Expects a JSON string to be deserialized as:\n```typescript\n{\nq?: string,\nsize?: number,\nbudget?: number\n}\n```', llm_chain=LLMChain(memory=None, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x104d8d250>, verbose=False, prompt=PromptTemplate(input_variables=['api_spec', 'question'], output_parser=None, partial_variables={}, template="Here is an API:\n\n{api_spec}\n\nYour job is to answer questions by returning valid JSON to se

In [4]:
from langchain.agents import Tool, AgentExecutor
from langchain.agents.agent import LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain

In [5]:
template = """Answer the following questions as best you can. You have access to the following tools (but you do not need to use them):

{tools}

If you want use a tool you should write valid TypeScript code to call it. Please put all parameters and their values on their own line. If you use strings, they should be wrapped in double quotes not single. Any arguments to this function should be values that can be loaded as json.

For your response, you have two options:

If you do not need to use any tools, you can just respond directly to the user by using the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Final Answer: your response to user here

If you do need to use tools, you should use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: ```typescript
...
``` 
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question.

When repsonding in your final answer to the user, they have NO knowledge of any intermediate steps. So if there is any intermediate knowledge there you want them to know, you should make sure to return it as part of your final answer.

Remember, you do not need to use tools. For example, if the user asks a question that you KNOW the answer to, you can respond. If they ask a question that you do not know the answer to, but no tool can help, you should respond asking for clarification.

Previous conversation (the question could be a follow up to something here):

{chat_history}

Begin!

Question: {input}
Thought: {agent_scratchpad}"""

In [6]:
shopify = toolkits[0]

In [7]:
class CustomPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        kwargs = self._merge_partial_and_user_variables(**kwargs)
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        return template.format(**kwargs)

In [8]:
anthropic_template = """\n\nHuman: Answer the following questions as best you can. You have access to the following tools (but you do not need to use them):

{tools}

If you want use a tool you should write valid TypeScript code to call it. Please put all parameters and their values on their own line. If you use strings, they should be wrapped in double quotes not single. Any arguments to this function should be values that can be loaded as json.

For your response, you have two options:

If you do not need to use any tools, you can just respond directly to the user by using the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Final Answer: your response to user here

If you do need to use tools, you should use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: ```typescript
...
``` 
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question.

When repsonding in your final answer to the user, they have NO knowledge of any intermediate steps. So if there is any intermediate knowledge there you want them to know, you should make sure to return it as part of your final answer.

Remember, you do not need to use tools. For example, if the user asks a question that you KNOW the answer to, you can respond. If they ask a question that you do not know the answer to, but no tool can help, you should respond asking for clarification.

Previous conversation (the question could be a follow up to something here):

{chat_history}

Begin!

Question: {input}\n\nAssistant: Thought:{agent_scratchpad}"""
anthropic_template1 = """{chat_history}\n\nHuman: "You are a helpful assistant. A very smart language model.

You have access to the following tools. The tools are TypeScript functions.

{tools}

If you want use a tool you should write valid TypeScript code to call it, always saving the output to a const with name "langchain". You should only make one function call at a time. Any arguments to this function should be values that can be loaded as json - e.g. no infinity values, etc. Any fuction calls MUST abide by the typescript definitions listed above.

Return a helpful response to the user. If you need to call a tool to get more information, just write the code for the tool in between a [TOOL_USE_BEGIN] and [TOOL_USE_END] token markers. You will get a response from the tool in between a [TOOL_RESULT_BEGIN] and [TOOL_RESULT_END] tokens. 
The user you are are interacting with will not see the Tool invocation or the Tool repsonse. Therefor, if there is information there that answers their question, you must relay that information to the user.

An example interaction may look like:

<Begin Example>
Human: What's the weather today?
Assistant: [TOOL_USE_BEGIN]const langchain = weather.onDate({{query: "1/2/2021"}})[TOOL_USE_END][TOOL_RESULT_BEGIN]49 degrees[TOOL_RESULT_END]The weather today is 49 degrees.
<End Example>

Begin!

Question: {input}\n\nAssistant:{agent_scratchpad}"""
class CustomAnthropicPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        kwargs = self._merge_partial_and_user_variables(**kwargs)
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"[TOOL_USE_END][TOOL_RESULT_BEGIN]{observation}[TOOL_RESULT_END]"
        kwargs["agent_scratchpad"] = thoughts
        return anthropic_template1.format(**kwargs)

In [9]:
anthropic_prompt = CustomAnthropicPromptTemplate(input_variables=["input", "intermediate_steps", "chat_history", "tools"])

In [10]:
prompt = CustomPromptTemplate(input_variables=["input", "intermediate_steps", "chat_history", "tools"])

In [11]:
PREFIX = """Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist."""


SUFFIX = """TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

{tools}

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me please, please output a response in one of two formats:

**Option 1:**
Use this if you want the human to use a tool. Output should be in a typescript Markdown snippet.
If you want use a tool you should write valid TypeScript code to call it. Please put all parameters and their values on their own line. If you use strings, they should be wrapped in double quotes not single. Any arguments to this function should be values that can be loaded as json.


```typescript
...

```

**Option #2:**
Use this if you want to respond directly to the human. Normal text formatted in the following schema:

Final Answer:

USER'S INPUT
--------------------
Here is the user's input (remember to respond as instructed):

{input}"""

TEMPLATE_TOOL_RESPONSE = """TOOL RESPONSE: 
---------------------
{observation}

USER'S INPUT
--------------------

Okay, so what is the response to my last comment? If using information obtained from the tools you must mention it explicitly without mentioning the tool names - I have forgotten all TOOL RESPONSES! Remember to respond as instructed originally."""
import json
from typing import Any, List, Optional, Sequence, Tuple

from langchain.agents.agent import Agent
from langchain.callbacks.base import BaseCallbackManager
from langchain.chains import LLMChain
from langchain.prompts.base import BasePromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    ChatPromptValue
)
from langchain.schema import (
    AgentAction,
    AIMessage,
    BaseLanguageModel,
    BaseMessage,
    BaseOutputParser,
    HumanMessage,
    SystemMessage,
    PromptValue,
)
from langchain.tools.base import BaseTool

class CustomChatPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs: Any) -> str:
        return self.format_prompt(**kwargs).to_string()
    
    def format_prompt(self, **kwargs: Any) -> PromptValue:
        kwargs = self._merge_partial_and_user_variables(**kwargs)
        result = [SystemMessage(content=PREFIX)] + kwargs["chat_history"]
        intermediate_steps = kwargs.pop("intermediate_steps")
        result.append(HumanMessage(content=SUFFIX.format(**kwargs)))
        for action, observation in intermediate_steps:
            result.append(AIMessage(content=action.log))
            human_message = HumanMessage(
                content=TEMPLATE_TOOL_RESPONSE.format(observation=observation)
            )
            result.append(human_message)
        return ChatPromptValue(messages=result)

In [12]:
BASE_CHAT_TEMPLATE = CustomChatPromptTemplate(input_variables=["chat_history", "intermediate_steps", "input", "tools"])

In [13]:
FINAL_ANSWER_ACTION = "Final Answer:"
import json
import json5
from langchain.schema import AgentAction, AgentFinish
import re
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output):
        if FINAL_ANSWER_ACTION in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split(FINAL_ANSWER_ACTION)[-1].strip()},
                log=llm_output,
            )
        parsed = llm_output.split("Action:")[1].split("```typescript")[1].split("```")[0].strip()
        if parsed.startswith("const"):
            parsed = parsed.split(" = ")[1]
        typescript_str = parsed.split("(")[1].split(")")[0]
        tool_input = json.dumps(json5.loads(typescript_str))
        return AgentAction(tool=parsed.split("(")[0], tool_input=tool_input, log=llm_output)

In [14]:
output_parser = CustomOutputParser()

In [15]:
class CustomOutputParser1(AgentOutputParser):
    
    def parse(self, llm_output):
        if "[TOOL_USE_BEGIN]" not in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.strip()},
                log=llm_output,
            )
        parsed = llm_output.split("[TOOL_USE_BEGIN]")[1].strip()
        if parsed.startswith("const"):
            parsed = parsed.split(" = ")[1]
        typescript_str = parsed.split("(")[1].split(")")[0]
        tool_input = json.dumps(json5.loads(typescript_str))
        return AgentAction(tool=parsed.split("(")[0], tool_input=tool_input, log=llm_output)

In [16]:
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

When answering, you must use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
<ACTION>the name of the action to take, should be one of [{tool_names}] listed above</ACTION>
<ACTION_INPUT>the natural language question to ask of this action</ACTION_INPUT>
Observation: the result of the action
... (this Thought/<ACTION>/<ACTION_INPUT>/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

{chat_history}
Question: {input}
{agent_scratchpad}"""

class CustomPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        
        kwargs = self._merge_partial_and_user_variables(**kwargs)
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"</ACTION_INPUT>\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        return template.format(**kwargs)

prompt = CustomPromptTemplate(input_variables=["input", "intermediate_steps", "tools", "tool_names", "chat_history"])

FINAL_ANSWER_ACTION = "Final Answer:"
from langchain.schema import AgentAction, AgentFinish
import re
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output):
        if FINAL_ANSWER_ACTION in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split(FINAL_ANSWER_ACTION)[-1].strip()},
                log=llm_output,
            )
        # \s matches against tab/newline/whitespace
        regex = r"<ACTION>(.*?)</ACTION>[\n]*<ACTION_INPUT>[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

## Test cases

In [17]:
from langchain.memory import ConversationBufferMemory
def get_agent(toolkits, llm):
#     if isinstance(llm, OpenAI):
#         prompt_to_use = prompt.partial(tools="\n\n".join([tk.get_typescript_namespace() for tk in toolkits]))
#         memory = ConversationBufferMemory(memory_key="chat_history", )
#         stop=["\nObservation:"]
#         output_parser = CustomOutputParser()
#     elif isinstance(llm, Anthropic):
#         prompt_to_use = anthropic_prompt.partial(tools="\n\n".join([tk.get_typescript_namespace() for tk in toolkits]))
#         memory = ConversationBufferMemory(memory_key="chat_history", )
#         stop=["[TOOL_USE_END]"]
#         output_parser = CustomOutputParser1()
#     elif isinstance(llm, ChatOpenAI):
#         prompt_to_use = BASE_CHAT_TEMPLATE.partial(tools="\n\n".join([tk.get_typescript_namespace() for tk in toolkits]))
#         memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
#         stop=["\nObservation:"]
#         output_parser = CustomOutputParser()
#     else:
#         raise ValueError
    tool_string = ""
    tool_name_list = []
    for tk in toolkits:
        for tool in tk.get_tools():
            tool_string += f"Name: {tool.name}\nDescription: {tool.description}\n\n"
            tool_name_list.append(tool.name)
    #prompt_to_use = anthropic_prompt.partial(tools="\n\n".join([tk.get_typescript_namespace() for tk in toolkits]))
    prompt_to_use = prompt.partial(tools=tool_string)
    prompt_to_use = prompt_to_use.partial(tool_names=",".join(tool_name_list))
    memory = ConversationBufferMemory(memory_key="chat_history", )
    stop=["</ACTION_INPUT>"]
    output_parser = CustomOutputParser()
    llm_chain = LLMChain(llm=llm, prompt=prompt_to_use)
    agent = LLMSingleActionAgent(llm_chain=llm_chain, output_parser=output_parser, stop=stop)
    
    tools = []
    for tk in toolkits:
        tools.extend(tk.get_tools())
    return AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)


In [18]:
MEMORY_TEST_CASES = [
    # Should know the answer to "whats my name"
    [
        {"input": "hi im bob"},
        {"input": "whats my name?"}
    ],
    # Should not know the answer to "whats my name"
    [
        {"input": "whats my name?"}
    ],
]
KLARNA_TEST_CASES = [
    
    [
        {"input": "what are some blue shirts you have available?"},
    ],
    [
        {"input": "what is the most expensive t-shirt you have?"},
        {"input": "whats the price in dollars?"},
    ],
]
MILO_TEST_CASES = [
    [
        {"input": "whats magic today?"},
        {"input": "thanks!"}
    ]
]

In [19]:
TESTS = [
    {
        "urls":[
            "https://www.klarna.com/.well-known/ai-plugin.json"
        ],
        "test_cases": KLARNA_TEST_CASES
    }
#     {
#         "urls":[
#             "https://www.joinmilo.com/.well-known/ai-plugin.json"
#         ],
#         "test_cases": MILO_TEST_CASES
#     }
]

In [20]:
from langchain.llms import OpenAI, Anthropic
from langchain.chat_models import ChatOpenAI
llm = OpenAI(temperature=0, verbose=True)
llm = ChatOpenAI(temperature=0)
#llm = Anthropic(temperature=0)

In [21]:
for test in TESTS:
    toolkits = get_toolkits(test["urls"])
    for tc in test["test_cases"]:
        agent = get_agent(toolkits, llm)
        for entry in tc:
            foo = agent(entry)



> Entering new AgentExecutor chain...


ValueError: Could not parse LLM output: `Thought: The person wants to know about available blue shirts from Klarna
<KlarnaProducts___productsUsingGET> 
<ACTION_INPUT>What blue shirt products do you have?`

In [ ]:
agent.agent.

In [ ]:
CustomOutputParser().parse(llm_output)

In [ ]:
%debug

In [ ]:
print(agent.agent.llm_chain.prompt.format(input="q", intermediate_steps="", chat_history=""))

In [43]:
import requests
url = "https://www.klarna.com/us/shopping/public/openai/v0/products"
params={
    "q": "t-shirt",
    "size": 1,
    "budget": 0
}

In [44]:
response = requests.get(url, params=params)

In [45]:
response

<Response [200]>

In [47]:
full_url = url + "?" + urllib.parse.urlencode({
    "q": "t-shirt",
    "size": 1,
    "budget": 0
})

In [ ]:
response = requests.get(full_url)

In [37]:
response.reason

'Bad Request'

In [ ]:
print(toolkits[0].get_tools()[0].description)

In [ ]:
from langchain.schema import BaseOutputParser
import json
class CustomOutputParser(BaseOutputParser):
    
    def parse(self, llm_output):
        parsed = llm_output.split("```json")[1].split("```")[0].strip()
        return json.loads(parsed)

In [ ]:
from langchain.llms import OpenAI, Anthropic
llm = Anthropic(temperature=0)

from langchain.chains.openapi_chain import OpenAPIChain
from langchain.requests import RequestsWrapper

In [ ]:
chain = OpenAPIChain.from_llm(llm, """API for fetching Klarna product information
Expects a JSON string to be deserialized as:
```typescript
{
q?: string,
size?: number,
budget?: number
}
```""", CustomOutputParser(), url="https://www.klarna.com/us/shopping/public/openai/v0/products",
    requests_method = "get",
    requests_wrapper= RequestsWrapper())

In [ ]:
chain.run("give me 2 blue shirts")

In [ ]:
prompt = """Here is an API:

API for fetching Klarna product information
Expects a JSON string to be deserialized as:
```typescript
{
q?: string,
size?: number,
budget?: number
}
```

Your job is to answer questions by returning valid JSON to send to this API in order to answer the user's question. 
Response with valid markdown, eg in the format:

[JSON BEGIN]
```json
...
```
[JSON END]

Here is the question you are trying to answer:

what is the most expensive t-shirt you have?"""

In [ ]:
llm(prompt, stop=["[JSON END]"])

In [ ]:
from langchain.

In [ ]:
from lang